In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2023-04-01'
final_date = '2023-04-30'
query_error_year = 2023
query_error_month = 4

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2023\04. ABRIL\DATA\ERROR ABRIL 2023.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 7)
#                 & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 7 )
#             & (query_error['year'] == 2022), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(125240, 17)

In [5]:
query_error['mes_cierre'].unique()

array([nan, 'Actual'], dtype=object)

In [6]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [7]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [8]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [9]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_17748\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
114510,114510,86f663aa-8670-4195-bb17-f9b739a30cb3,NaN,AM564SFM5,2023-04-01,431.21,ERROR,Bank account information not available,2023-04-01,4,2023,NEW,SIN_CLABE,86f663aa-8670-4195-bb17-f9b739a30cb3,2023-04-01,1,Actual
114468,114468,6fb8bc6b-93d5-40f5-b374-f22d84493e5b,NaN,A5ZG35KPW,2023-04-01,239.56,ERROR,Bank account information not available,2023-04-01,4,2023,NEW,SIN_CLABE,6fb8bc6b-93d5-40f5-b374-f22d84493e5b,2023-04-01,1,Actual
114467,114467,fddb3db6-64da-4539-95e7-5c8a31a82855,NaN,AWPT56CYY,2023-04-01,0.02,ERROR,Bank account information not available,2023-04-01,4,2023,NEW,SIN_CLABE,fddb3db6-64da-4539-95e7-5c8a31a82855,2023-04-01,1,Actual
114466,114466,a27d5fc0-7749-498a-9933-7cee627b5555,NaN,A2WDNBPQ8,2023-04-01,0.02,ERROR,Bank account information not available,2023-04-01,4,2023,NEW,SIN_CLABE,a27d5fc0-7749-498a-9933-7cee627b5555,2023-04-01,1,Actual
114465,114465,ed642c2c-5451-418c-abd8-c7994267cf56,NaN,A7LVFXLWM,2023-04-01,0.95,ERROR,Bank account information not available,2023-04-01,4,2023,NEW,SIN_CLABE,ed642c2c-5451-418c-abd8-c7994267cf56,2023-04-01,1,Actual


In [10]:
query_error_current_month.shape

(10972, 17)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# AGRUPADO POR ESQUEMA

In [14]:
group_esquema = pd.pivot_table(query_error_current_month,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only      amount                          \
esquema_query                      ERROR_USUAL               SIN_CLABE   
status_cubeta                            ERROR          NEW        NEW   
0                       2023-04-01        0.00     9,580.50  79,183.93   
1                       2023-04-02        0.00     1,210.03 101,684.18   
2                       2023-04-03    2,988.27    19,129.95  90,778.85   
3                       2023-04-04        0.00 1,038,043.60  80,323.25   
4                       2023-04-05   15,626.48   836,402.44  60,610.03   

                            
esquema_query          All  
status_cubeta               
0                88,764.43  
1               102,894.21  
2               112,897.07  
3             1,118,366.85  
4               912,638.95

In [15]:
group_esquema.shape

(31, 5)

In [16]:
writer = pd.ExcelWriter('FASE VI. DATA POLIZA ERROR-SIN CLABE ABRIL 2023.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
query_error_current_month.to_excel(writer, sheet_name='DETALLE',index=False)
writer.close()